In [ ]:
import re
import json
import os
from deep_translator import GoogleTranslator

# 번역기 설정
translator = GoogleTranslator(source='en', target='ko')

# HTML 태그 보존하며 번역하는 함수
def translate_text(text):
    if not text or not isinstance(text, str):
        return text

    # &amp;Reference[...] 보호 → 임시 토큰으로 변경
    reference_map = {}
    def ref_replacer(match):
        key = f"__REF_{len(reference_map)}__"
        reference_map[key] = match.group(0)
        return key

    text = re.sub(r'@UUID\[.*?\]{.*?}', lambda m: m.group(0), text)  # UUID 그대로 유지
    text = re.sub(r'&amp;Reference\[.*?\]', ref_replacer, text)

    # HTML 태그 분리
    parts = re.split(r'(<[^>]+>)', text)
    translated_parts = []

    for part in parts:
        if re.match(r'<[^>]+>', part) or not part.strip():
            translated_parts.append(part)
        else:
            try:
                translated = translator.translate(part)
                translated_parts.append(translated if translated else part)
            except Exception:
                translated_parts.append(part)

    result = ''.join(translated_parts)

    # 번역 후 임시 토큰 복구
    for key, original in reference_map.items():
        result = result.replace(key, original)

    return result


# name/title은 "한글(영어)" 형태
def translate_name_or_title(value):
    try:
        translated = translator.translate(value)
        return f"{translated}({value})" if translated else value
    except Exception:
        return value

# 재귀적으로 모든 필드를 순회하며 번역
def recursive_translate(data):
    if isinstance(data, dict):
        translated = {}
        for k, v in data.items():
            if k in ['name', 'title']:
                translated[k] = translate_name_or_title(v)
            elif k in ['description', 'hint', 'condition']:
                translated[k] = translate_text(v)
            else:
                translated[k] = recursive_translate(v)
        return translated
    elif isinstance(data, list):
        return [recursive_translate(item) for item in data]
    elif isinstance(data, str):
        return data
    else:
        return data

# 입력 및 출력 경로
input_path = "./aa.json"
output_path = "./aa.translated.json"

# JSON 로드
with open(input_path, "r", encoding="utf-8") as f:
    source_json = json.load(f)

# 번역 수행
translated_json = recursive_translate(source_json)

# 저장 (경로 없으면 생성)
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_json, f, ensure_ascii=False, indent=2)

print("✅ 번역 및 저장 완료!")


✅ 번역 및 저장 완료!
